In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle


import os
import sys
import annoy
from annoy import AnnoyIndex
import random



from IPython.display import display_markdown

from collections import Counter

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial import cKDTree
from sklearn.decomposition import PCA
from sklearn.decomposition import SparsePCA


import antares_client

import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# set random seed for consistent results
import random
random.seed(0)

plt.style.use('fig_publication.mplstyle')

%config InlineBackend.figure_format = 'retina'  #for MacOS, make plots crisp

In [2]:
fakes_suffix = 'SMOTE_train_test_70-30_min14_kneighbors8' # really smote_suffix 
suffix=f"noTNScut_pca60_noCadence_NOtransf_HasHostGalOnly_SS_AlerceCls_NOfw_{fakes_suffix}"

In [3]:
full_ann_properties_df = pd.read_csv(f'../loci_dbs/alerce_cut/full_ann101_properties_df_nofakes_{suffix}.csv', index_col=0)
full_ann_properties_df

,ann=0_ztfid,ann=1_ztfid,ann=2_ztfid,ann=3_ztfid,ann=4_ztfid,ann=5_ztfid,ann=6_ztfid,ann=7_ztfid,ann=8_ztfid,ann=9_ztfid,...,ann=91_spec_z,ann=92_spec_z,ann=93_spec_z,ann=94_spec_z,ann=95_spec_z,ann=96_spec_z,ann=97_spec_z,ann=98_spec_z,ann=99_spec_z,ann=100_spec_z
ZTF20aaybwth,ZTF20aaybwth,ZTF21abexmbi,ZTF21aalxxsn,ZTF18aavaueu,ZTF20acxyrkk,ZTF21aazgkjf,ZTF20abzcefc,ZTF21aausvhf,ZTF20acnvmbp,ZTF18aawgrxz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21abhvfrj,ZTF21abhvfrj,ZTF21aarlqgt,ZTF18ablqjws,ZTF21aaabvzf,ZTF18accwihs,ZTF20acvhwlp,ZTF21achypsm,ZTF20acwkbjm,ZTF20acuxlgx,ZTF21abcowbx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF20acngnvf,ZTF20acngnvf,ZTF21aahdrvq,ZTF20acusjnl,ZTF21acjeqpb,ZTF18acttqzo,ZTF20aansxav,ZTF19abeievz,ZTF21achqnfn,ZTF21achnqdo,ZTF20aaomtcc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF20aciucub,ZTF20aciucub,ZTF18abscghc,ZTF21aavouex,ZTF21aatuuiz,ZTF21aawjiss,ZTF21aagkzxe,ZTF21aagojoi,ZTF21abtoqij,ZTF21aagxvsx,ZTF21abcxfhj,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21abcfnuk,ZTF21abcfnuk,ZTF21aagrhot,ZTF21aarzrnb,ZTF21abalkop,ZTF21abfyzry,ZTF21aahpakt,ZTF21aakyrok,ZTF21abvwuyk,ZTF21aarhgns,ZTF20acoheud,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF20acupxyy,ZTF20acupxyy,ZTF21aaoglbq,ZTF21abawvgt,ZTF21aabgjcz,ZTF21aayoneu,ZTF20acpiahx,ZTF21aabiipy,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21aarzxnb,ZTF21aarzxnb,ZTF21abdhbrh,ZTF20acpmeuc,ZTF21acjvkga,ZTF21aawkuqe,ZTF20acuxlgx,ZTF20acppzxx,ZTF18acbzvow,ZTF21aaigndt,ZTF21abbgtcy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF18acsxjhc,ZTF18acsxjhc,ZTF18aczersc,ZTF18aakycpv,ZTF19aabyqei,ZTF18aahjdqr,ZTF18acryhnx,ZTF20aaeaftf,ZTF18abugmrg,ZTF18acrygry,ZTF19aauojsf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21aatklhw,ZTF21aatklhw,ZTF21absvjjq,ZTF20ackkejs,ZTF21aawaguv,ZTF21abrzdqx,ZTF21aaobcrz,ZTF20acowuix,ZTF21aapqzxr,ZTF20acnsinb,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
full_ann_has_tns_df = full_ann_properties_df.iloc[:, 202:303]
full_ann_has_tns_df

,ann=0_has_tns,ann=1_has_tns,ann=2_has_tns,ann=3_has_tns,ann=4_has_tns,ann=5_has_tns,ann=6_has_tns,ann=7_has_tns,ann=8_has_tns,ann=9_has_tns,...,ann=91_has_tns,ann=92_has_tns,ann=93_has_tns,ann=94_has_tns,ann=95_has_tns,ann=96_has_tns,ann=97_has_tns,ann=98_has_tns,ann=99_has_tns,ann=100_has_tns
ZTF20aaybwth,Yes_TNS,Yes_TNS,Yes_TNS,No_TNS,No_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21abhvfrj,Yes_TNS,Yes_TNS,No_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF20acngnvf,No_TNS,Yes_TNS,No_TNS,Yes_TNS,No_TNS,No_TNS,No_TNS,Yes_TNS,Yes_TNS,No_TNS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF20aciucub,Yes_TNS,No_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,No_TNS,Yes_TNS,Yes_TNS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21abcfnuk,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF20acupxyy,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21aarzxnb,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,No_TNS,Yes_TNS,Yes_TNS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF18acsxjhc,Yes_TNS,No_TNS,Yes_TNS,Yes_TNS,Yes_TNS,No_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21aatklhw,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
full_ann_tns_cls_df = full_ann_properties_df.iloc[:, 303:404]
full_ann_tns_cls_df

,ann=0_tns_cls,ann=1_tns_cls,ann=2_tns_cls,ann=3_tns_cls,ann=4_tns_cls,ann=5_tns_cls,ann=6_tns_cls,ann=7_tns_cls,ann=8_tns_cls,ann=9_tns_cls,...,ann=91_tns_cls,ann=92_tns_cls,ann=93_tns_cls,ann=94_tns_cls,ann=95_tns_cls,ann=96_tns_cls,ann=97_tns_cls,ann=98_tns_cls,ann=99_tns_cls,ann=100_tns_cls
ZTF20aaybwth,None,None,None,None,None,SN IIn,SN II,None,None,SN II,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21abhvfrj,None,SN II,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF20acngnvf,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF20aciucub,None,None,None,None,SN Ia,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21abcfnuk,None,SN Ia,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF20acupxyy,None,None,None,None,None,None,TDE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21aarzxnb,None,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF18acsxjhc,None,None,SN IIn,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21aatklhw,SN Ia,None,SN Ia,None,SN Ia,SN Ia,SN Ia,SN Ia,SN Ia,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
full_ann_spec_z_df = full_ann_properties_df.iloc[:, 404:505]
full_ann_spec_z_df

,ann=0_spec_z,ann=1_spec_z,ann=2_spec_z,ann=3_spec_z,ann=4_spec_z,ann=5_spec_z,ann=6_spec_z,ann=7_spec_z,ann=8_spec_z,ann=9_spec_z,...,ann=91_spec_z,ann=92_spec_z,ann=93_spec_z,ann=94_spec_z,ann=95_spec_z,ann=96_spec_z,ann=97_spec_z,ann=98_spec_z,ann=99_spec_z,ann=100_spec_z
ZTF20aaybwth,-99.00000,-99.000,-99.00000,-99.0,-99.000,0.142,0.056,-99.0000,-99.000,0.0267,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21abhvfrj,-99.00000,0.067,-99.00000,-99.0,-99.000,-99.000,-99.000,-99.0000,-99.000,-99.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF20acngnvf,-99.00000,-99.000,-99.00000,-99.0,-99.000,-99.000,-99.000,-99.0000,-99.000,-99.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF20aciucub,-99.00000,-99.000,-99.00000,-99.0,0.130,-99.000,-99.000,-99.0000,-99.000,-99.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21abcfnuk,-99.00000,0.100,-99.00000,-99.0,-99.000,-99.000,-99.000,-99.0000,-99.000,-99.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF20acupxyy,-99.00000,-99.000,-99.00000,-99.0,-99.000,-99.000,0.152,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21aarzxnb,-99.00000,-99.000,-99.00000,-99.0,-99.000,-99.000,-99.000,-99.0000,-99.000,-99.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF18acsxjhc,-99.00000,-99.000,0.04600,-99.0,-99.000,-99.000,-99.000,-99.0000,-99.000,-99.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21aatklhw,0.06689,-99.000,0.09171,-99.0,0.066,0.060,0.090,0.1035,0.090,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
full_ann_tns_cls_spec_df = full_ann_tns_cls_df[full_ann_tns_cls_df['ann=0_tns_cls'] != 'None'] # has spec class in test set
full_ann_tns_cls_spec_df

,ann=0_tns_cls,ann=1_tns_cls,ann=2_tns_cls,ann=3_tns_cls,ann=4_tns_cls,ann=5_tns_cls,ann=6_tns_cls,ann=7_tns_cls,ann=8_tns_cls,ann=9_tns_cls,...,ann=91_tns_cls,ann=92_tns_cls,ann=93_tns_cls,ann=94_tns_cls,ann=95_tns_cls,ann=96_tns_cls,ann=97_tns_cls,ann=98_tns_cls,ann=99_tns_cls,ann=100_tns_cls
ZTF21aazebgh,SN Ia,SN Ia,None,SN Ia,SN Ia,None,SN Ia,None,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21acgxypn,SN Ic-BL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF19aapqqrb,SN Ia,None,None,SN Ia,None,SN Ia,SN Ia,None,SN Ia,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21aasbxng,SN Ia,SN Ia,SN Ia,SN Ia,SN Ia,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21aaglgcb,SN Ia,SN Ia,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF21acdalyp,SN Ia-91T-like,SN Ia,SN Ia,SN Ia,SN Ia,SN II,SN Ia,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21abdcauj,SN Ia,SN Ia,SN Ia,SN Ia,None,None,None,SN Ia,SN Ia,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21aaetnfe,SN Ia,SN Ia,SN Ia,SN Ia,SN Ia,SN Ia,SN Ia,SN Ia,SN Ia,SN Ia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21acgzwsc,SN Ia,SN Ia,SN IIb,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
full_ann_ztfid_spec_df = full_ann_properties_df[full_ann_properties_df['ann=0_tns_cls'] != 'None'].iloc[:, 0:101]
full_ann_ztfid_spec_df

,ann=0_ztfid,ann=1_ztfid,ann=2_ztfid,ann=3_ztfid,ann=4_ztfid,ann=5_ztfid,ann=6_ztfid,ann=7_ztfid,ann=8_ztfid,ann=9_ztfid,...,ann=91_ztfid,ann=92_ztfid,ann=93_ztfid,ann=94_ztfid,ann=95_ztfid,ann=96_ztfid,ann=97_ztfid,ann=98_ztfid,ann=99_ztfid,ann=100_ztfid
ZTF21aazebgh,ZTF21aazebgh,ZTF21aasrdct,ZTF21aauaemd,ZTF21abyonuw,ZTF21aaxyecd,ZTF21abdbtqw,ZTF21abasxdp,ZTF21abhzsho,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21acgxypn,ZTF21acgxypn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF19aapqqrb,ZTF19aapqqrb,ZTF18adaknbe,ZTF21aarwgjb,ZTF18aaizgoq,ZTF21aaarhls,ZTF20acobevy,ZTF21aaaekuh,ZTF20abmkwxg,ZTF21abovblm,ZTF21aaxsosp,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21aasbxng,ZTF21aasbxng,ZTF21abgzlyi,ZTF18aabeszt,ZTF20acmzoxo,ZTF20acpumut,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21aaglgcb,ZTF21aaglgcb,ZTF20ackitai,ZTF20acpayvy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF21acdalyp,ZTF21acdalyp,ZTF21abrghol,ZTF20acmzoxo,ZTF21abhaqju,ZTF20acrdemq,ZTF18aadsuxd,ZTF21aapowty,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21abdcauj,ZTF21abdcauj,ZTF21abkqkon,ZTF21abrotmc,ZTF21abqueax,ZTF21abhzsho,ZTF18aasvfah,ZTF20acrjjse,ZTF20acoaqpm,ZTF20aciugdl,ZTF20acowutp,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21aaetnfe,ZTF21aaetnfe,ZTF21acafwof,ZTF21aaquiwu,ZTF21aafgljq,ZTF21acbmxgn,ZTF20acywbes,ZTF21acdvkcj,ZTF21abvcarc,ZTF18abavruc,ZTF21abdvcaf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21acgzwsc,ZTF21acgzwsc,ZTF18aabeszt,ZTF21aanrhag,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
full_ann_has_tns_spec_df = full_ann_has_tns_df[full_ann_has_tns_df.index.isin(full_ann_ztfid_spec_df.index)]
full_ann_has_tns_spec_df

,ann=0_has_tns,ann=1_has_tns,ann=2_has_tns,ann=3_has_tns,ann=4_has_tns,ann=5_has_tns,ann=6_has_tns,ann=7_has_tns,ann=8_has_tns,ann=9_has_tns,...,ann=91_has_tns,ann=92_has_tns,ann=93_has_tns,ann=94_has_tns,ann=95_has_tns,ann=96_has_tns,ann=97_has_tns,ann=98_has_tns,ann=99_has_tns,ann=100_has_tns
ZTF21aazebgh,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21acgxypn,Yes_TNS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF19aapqqrb,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21aasbxng,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21aaglgcb,Yes_TNS,Yes_TNS,Yes_TNS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF21acdalyp,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21abdcauj,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21aaetnfe,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,Yes_TNS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZTF21acgzwsc,Yes_TNS,Yes_TNS,Yes_TNS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# iterate over the rows of both dataframes simultaneously
unique_missed_SNe_cand_l = []
for (index1, row1), (index2, row2), (index3, row3) in zip(full_ann_ztfid_spec_df.iterrows(), 
                                          full_ann_has_tns_spec_df.iterrows(),
                                          full_ann_tns_cls_spec_df.iterrows()):
    print(f"Starting ZTF index: {index1}, {row3['ann=0_tns_cls']}") 
    for ann, (ztf, tns) in enumerate(zip(row1, row2)):
        if isinstance(ztf, str): 
            if tns == 'No_TNS':
                print(f"https://alerce.online/object/{ztf} | ANN={ann}")
                unique_missed_SNe_cand_l.append(ztf)
        else: continue

unique_missed_SNe_cand_l = np.unique(unique_missed_SNe_cand_l)

Starting ZTF index: ZTF21aazebgh, SN Ia
Starting ZTF index: ZTF21acgxypn, SN Ic-BL
Starting ZTF index: ZTF19aapqqrb, SN Ia
Starting ZTF index: ZTF21aasbxng, SN Ia
Starting ZTF index: ZTF21aaglgcb, SN Ia
Starting ZTF index: ZTF21aanwfjk, SN Ia
Starting ZTF index: ZTF21abcixor, SN II
Starting ZTF index: ZTF21abgkfzh, SN IIn
Starting ZTF index: ZTF21aarerya, SN Ia
https://alerce.online/object/ZTF18acueeoo | ANN=8
https://alerce.online/object/ZTF20acusjnl | ANN=12
https://alerce.online/object/ZTF20acxyrkk | ANN=17
https://alerce.online/object/ZTF20abmbvyu | ANN=20
https://alerce.online/object/ZTF18absmsbm | ANN=23
https://alerce.online/object/ZTF20acrssoi | ANN=27
Starting ZTF index: ZTF21abvvuhm, SN Ia
Starting ZTF index: ZTF21acivpbd, SN II
Starting ZTF index: ZTF21abjtqyq, SN Ia
Starting ZTF index: ZTF21abotose, SN IIb
https://alerce.online/object/ZTF18aaluviz | ANN=1
https://alerce.online/object/ZTF19aapfxij | ANN=6
https://alerce.online/object/ZTF20acdglxa | ANN=7
https://alerce.onlin

In [50]:
for mSne in unique_missed_SNe_cand_l:
    print(f"https://alerce.online/object/{mSne}")

https://alerce.online/object/ZTF18aajgowk
https://alerce.online/object/ZTF18aajkgtr
https://alerce.online/object/ZTF18aajmlnp
https://alerce.online/object/ZTF18aaluviz
https://alerce.online/object/ZTF18aaytijy
https://alerce.online/object/ZTF18aayyedm
https://alerce.online/object/ZTF18ablqjws
https://alerce.online/object/ZTF18abnysyy
https://alerce.online/object/ZTF18abnzney
https://alerce.online/object/ZTF18abpucqe
https://alerce.online/object/ZTF18abrxvpd
https://alerce.online/object/ZTF18abskrcm
https://alerce.online/object/ZTF18absljwl
https://alerce.online/object/ZTF18absmsbm
https://alerce.online/object/ZTF18abtjrbt
https://alerce.online/object/ZTF18abuioue
https://alerce.online/object/ZTF18abuluko
https://alerce.online/object/ZTF18abuvqgo
https://alerce.online/object/ZTF18acanabv
https://alerce.online/object/ZTF18acbvijf
https://alerce.online/object/ZTF18acbwgmi
https://alerce.online/object/ZTF18acbzvzm
https://alerce.online/object/ZTF18acchzlc
https://alerce.online/object/ZTF18

In [51]:
len(unique_missed_SNe_cand_l)

106

## ^Some are AGN (maybe 20%), the rest are missed SNe (mostly Ia)!

### Interesting Missed SNe besides SN Ia (Indeed NOT in TNS!!)

https://alerce.online/object/ZTF21aaualdi - missed SLSN! (ANN=34 of ZTF21acizbmv, SN Ia)
https://alerce.online/object/ZTF21aasdcgt - missed SLSN/TDE! (ANN=15 of the same ZTF21acizbmv, SN Ia)
https://alerce.online/object/ZTF20acrssoi - missed IIn/SLSN
https://alerce.online/object/ZTF18acvgmpx - SN II in bright spiral gal closeby ish (but in 2018 so makes sense it was missed)
Some more SN IIs

# Look at all No_TNS in dataset...

In [52]:
bank_and_tns_df = pd.read_csv('../loci_dbs/alerce_cut/bank_and_tns_df.csv.gz', compression='gzip', index_col=0)
bank_and_tns_df

,feature_amplitude_magn_r,feature_anderson_darling_normal_magn_r,feature_beyond_1_std_magn_r,feature_beyond_2_std_magn_r,feature_cusum_magn_r,feature_inter_percentile_range_2_magn_r,feature_inter_percentile_range_10_magn_r,feature_inter_percentile_range_25_magn_r,feature_kurtosis_magn_r,feature_linear_fit_slope_magn_r,...,rApMag_rKronMag,iApMag_iKronMag,zApMag_zKronMag,yApMag_yKronMag,7DCD,dist/DLR,has_tns,tns_cls,spec_z,report_group
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF17aaahrni,1.501946,0.581482,0.333333,0.000000,0.300476,3.003893,2.881442,1.473296,-0.323815,0.000616,...,-0.071001,0.051300,-0.024000,-0.145601,22.882911,2.639232,Yes_TNS,None,-99.00,GaiaAlerts
ZTF17aadqidc,0.118791,0.276625,0.428571,0.000000,0.385504,0.237581,0.223594,0.138365,-0.817505,0.013288,...,0.163198,0.160099,0.150101,0.122200,26.155457,0.051662,No_TNS,None,-99.00,None
ZTF17aaducdh,0.384243,0.400104,0.333333,0.083333,0.379044,0.768486,0.586268,0.218094,0.425088,0.025251,...,0.223600,0.021301,0.269098,0.174500,11.297927,0.018901,Yes_TNS,None,-99.00,GaiaAlerts
ZTF18aaaokus,1.337077,4.369050,0.320225,0.005618,0.365946,2.108598,1.640040,0.837900,-0.695091,-0.001890,...,1.217501,1.251801,1.243600,1.272200,119.727409,0.005603,Yes_TNS,None,-99.00,ZTF
ZTF18aaavffc,1.251362,2.169584,0.291667,0.025000,0.338697,2.387948,2.022931,0.747013,-0.276298,-0.002842,...,0.625299,0.541301,0.607000,0.500100,87.296246,0.003534,Yes_TNS,None,-99.00,ALeRCE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF22aaboeex,0.068989,0.268161,0.500000,0.000000,0.381361,0.137978,0.132096,0.079157,-0.901772,-0.028012,...,0.708401,0.683500,0.644199,0.548500,25.406284,1.134230,Yes_TNS,SN Ia,0.09,ZTF
ZTF22aaboiqa,0.205293,0.126299,0.400000,0.000000,0.364903,0.410585,0.410585,0.253444,-1.169788,-0.029534,...,-0.035500,-0.026699,0.067301,-0.125101,10.365118,0.148034,Yes_TNS,None,-99.00,ANTARES
ZTF22aabovlw,0.113249,0.078036,0.500000,0.000000,0.396465,0.226498,0.226498,0.157836,-1.919301,0.000368,...,0.606100,0.483999,0.585100,0.647001,33.811442,4.407910,Yes_TNS,None,-99.00,ANTARES


In [54]:
all_obj_no_tns_df = bank_and_tns_df[(bank_and_tns_df.tns_cls == "None") & (bank_and_tns_df.has_tns == "No_TNS")]
all_obj_no_tns_df # Some missed SNe, some AGN, some bogus...

,feature_amplitude_magn_r,feature_anderson_darling_normal_magn_r,feature_beyond_1_std_magn_r,feature_beyond_2_std_magn_r,feature_cusum_magn_r,feature_inter_percentile_range_2_magn_r,feature_inter_percentile_range_10_magn_r,feature_inter_percentile_range_25_magn_r,feature_kurtosis_magn_r,feature_linear_fit_slope_magn_r,...,rApMag_rKronMag,iApMag_iKronMag,zApMag_zKronMag,yApMag_yKronMag,7DCD,dist/DLR,has_tns,tns_cls,spec_z,report_group
ztf_object_id,,,,,,,,,,,,,,,,,,,,,
ZTF17aadqidc,0.118791,0.276625,0.428571,0.000000,0.385504,0.237581,0.223594,0.138365,-0.817505,0.013288,...,0.163198,0.160099,0.150101,0.122200,26.155457,0.051662,No_TNS,None,-99.0,None
ZTF18aabqqwb,1.313333,0.666632,0.444444,0.000000,0.421515,2.626665,2.405153,1.837282,-1.445592,-0.000028,...,-0.141600,-0.109600,-0.052898,-0.119699,5.441947,25.049507,No_TNS,None,-99.0,None
ZTF18aabteyx,0.257150,2.547349,0.201258,0.062893,0.180521,0.336944,0.140364,0.067526,3.749270,0.000011,...,0.620899,0.796900,0.732100,0.952000,67.524753,0.636372,No_TNS,None,-99.0,None
ZTF18aaddhpb,1.321350,2.235420,0.216216,0.032432,0.274615,0.928559,0.520700,0.276745,14.181516,0.000387,...,-0.053598,-0.004700,-0.097601,-0.157000,33.069125,8.821872,No_TNS,None,-99.0,None
ZTF18aaghvyl,0.582516,0.971906,0.440367,0.009174,0.389432,1.042997,0.815100,0.486705,-0.976237,-0.000902,...,0.083599,0.127001,0.053202,0.012400,200.237178,0.060303,No_TNS,None,-99.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF21acjoxzv,0.169200,0.130463,0.250000,0.000000,0.350440,0.338400,0.338400,0.207150,1.567070,-0.005867,...,0.153700,0.225901,0.219299,0.279100,106.862017,0.063256,No_TNS,None,-99.0,None
ZTF21ackhfvp,0.165550,0.414492,0.250000,0.000000,0.307202,0.331100,0.287599,0.150997,0.451826,0.010731,...,0.026299,-0.187700,-0.063601,-0.207500,12.429270,0.087465,No_TNS,None,-99.0,None
ZTF21acmhpxe,0.301300,0.073711,0.500000,0.000000,0.375915,0.602600,0.602600,0.386220,-0.361028,0.010994,...,-0.158600,0.006399,-0.055199,-0.057199,5.883018,0.020985,No_TNS,None,-99.0,None


In [55]:
for obj in all_obj_no_tns_df.index:
    print(f"https://alerce.online/object/{obj}")

https://alerce.online/object/ZTF17aadqidc
https://alerce.online/object/ZTF18aabqqwb
https://alerce.online/object/ZTF18aabteyx
https://alerce.online/object/ZTF18aaddhpb
https://alerce.online/object/ZTF18aaghvyl
https://alerce.online/object/ZTF18aagpyca
https://alerce.online/object/ZTF18aahqqry
https://alerce.online/object/ZTF18aaiuynw
https://alerce.online/object/ZTF18aajgowk
https://alerce.online/object/ZTF18aajkgtr
https://alerce.online/object/ZTF18aajmlnp
https://alerce.online/object/ZTF18aajswpc
https://alerce.online/object/ZTF18aakeanj
https://alerce.online/object/ZTF18aalsudf
https://alerce.online/object/ZTF18aaluviz
https://alerce.online/object/ZTF18aamdjfh
https://alerce.online/object/ZTF18aamofne
https://alerce.online/object/ZTF18aamurqd
https://alerce.online/object/ZTF18aapbilo
https://alerce.online/object/ZTF18aaqlfeo
https://alerce.online/object/ZTF18aaqzdge
https://alerce.online/object/ZTF18aaswyuj
https://alerce.online/object/ZTF18aathofv
https://alerce.online/object/ZTF18

### Interesting Missed SNe from entire dataset

https://alerce.online/object/ZTF18aabteyx - template has SN
https://alerce.online/object/ZTF18aamdjfh - template has SN, hostless
https://alerce.online/object/ZTF18abiitmq - SN II, long rise
https://alerce.online/object/ZTF18abkhqww - SN IIP
https://alerce.online/object/ZTF18abkmfaj - SN II
https://alerce.online/object/ZTF18absljlb - SN II
https://alerce.online/object/ZTF18absljwl - SN II
https://alerce.online/object/ZTF18acvgmpx - SN II, closeby spiral
https://alerce.online/object/ZTF18acvwkvc - TDE? or AGN?
https://alerce.online/object/ZTF20aaznbiu - think AGN, but small chance it's SLSN
https://alerce.online/object/ZTF20abztvjb - SN II
https://alerce.online/object/ZTF20acbiwfi - long lived
https://alerce.online/object/ZTF20acbptqx - SLSN?
https://alerce.online/object/ZTF20acfamrq - SN II
https://alerce.online/object/ZTF20acipeqz - SN II
https://alerce.online/object/ZTF20acngnvf - SLSN (got to 19.5 mag and lasted almost 100d...)
https://alerce.online/object/ZTF20acnztaa - SN II, IIn
https://alerce.online/object/ZTF20acpgnmw - TDE?
https://alerce.online/object/ZTF20acpzjsk - TDE?, maybe AGN
https://alerce.online/object/ZTF20acrssoi - IIn, SLSN
https://alerce.online/object/ZTF20acxyrkk - IIn, SLSN
https://alerce.online/object/ZTF20acyroio - IIn, SLSN
https://alerce.online/object/ZTF20acyybza - IIP
https://alerce.online/object/ZTF20acyydhn - II
https://alerce.online/object/ZTF21aabyoxk - ?? Interesting! Like IIn but blue the whole way and way offset from center (not TDE). In elliptical at z=0.14. Off center TDE? Weird? something else??? Peak abs mag ~ -19.2...

In [63]:
# https://alerce.online/object/ZTF21aabyoxk
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(70,0.3)
z=0.14
mu = cosmo.distmod(z).value
peak_abs_mag = 19.9-mu
peak_abs_mag

-19.20248388428707